<a href="https://colab.research.google.com/github/akkipant/CS5560-Knowledge_Discovery_and_Management/blob/main/ICP-4/Source/KDM_ICP_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up PySpark in Colab

Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to download Java

In [1]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,929 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [

Next, we will install Apache Spark 3.0.1 with Hadoop 2.7

In [2]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

Now, we just need to unzip that folder.

In [3]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

There is one last thing that we need to install and that is the findspark library. It will locate Spark on the system and import it as a regular library.

In [4]:
!pip install -q findspark

Now that we have installed all the necessary dependencies in Colab, it is time to set the environment path. This will enable us to run Pyspark in the Colab environment.

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.

In [6]:
import findspark
findspark.init()

If you want to know the location where Spark is installed, use findspark.find()

In [7]:
findspark.find()

'/content/spark-3.0.1-bin-hadoop2.7'

Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.

You can give a name to the session using appName() and add some configurations with config() if you wish.

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Finally, print the SparkSession variable.

In [ ]:
spark

In [10]:
# load data from data.csv file

df = spark.read.csv("/content/data.csv", header=True, inferSchema= True)
df.show()


+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

# 1. Transformation Tasks

## a. Sorting data by Total Charges(Desc Order)

In [48]:
#task 1
transform_1 = df.sort(df.TotalCharges.desc())
print("Transformation 1: Sort data based on Total Charges in Descending Order")
transform_1.show()

Transformation 1: Sort data based on Total Charges in Descending Order
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+----------

## b. Filtering data for those who have Fiber optic

In [47]:
# task 2

transform_2 = df.filter(df['InternetService'] == 'Fiber optic')
print("Transformation 3: Filtering data based on Internet Service to include only FiberOptic users")
transform_2.show()

Transformation 3: Filtering data based on Internet Service to include only FiberOptic users
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|9237-HQITU|Female|            0|     No|        No|     2|         Yes|           

## c. Combining transformation task: Filtering + grouping by 

In [46]:
# task 3

# showing number of Male or Female users of DSL with monthly charge of 50 or less
transform_3 = df.filter('MonthlyCharges < 50 AND InternetService == "DSL"').groupBy('Gender').count()
print("Transformation 3: Filtering data based on MonthlyCharges and Service type which is then grouped by Gender")
transform_3.show()

Transformation 3: Filtering data based on MonthlyCharges and Service type which is then grouped by Gender
+------+-----+
|Gender|count|
+------+-----+
|Female|  362|
|  Male|  406|
+------+-----+



# 2. Actions Tasks 

## a. Action:  get first element using first function

In [34]:
# action task 1
action_1 = df.first()
print("Action 1: Print the first row")
action_1


Action 1: Print the first row


Row(customerID='7590-VHVEG', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='No', tenure=1, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineSecurity='No', OnlineBackup='Yes', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.85, TotalCharges='29.85', Churn='No')

## b. Action: Printing summary

In [44]:
# action task 2
action_2 = df.summary()
print("Action 2: Getting all the summary of data set")
action_2


Action 2: Getting all the summary of data set


DataFrame[summary: string, customerID: string, gender: string, SeniorCitizen: string, Partner: string, Dependents: string, tenure: string, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: string, TotalCharges: string, Churn: string]

## c. Action: Printing last 4 element

In [43]:
# action Task3

action_3 = df.tail(4)
print("Action 3: Showing first element of data set")
action_3

Action 3: Showing first element of data set


[Row(customerID='2234-XADUH', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='Yes', tenure=72, PhoneService='Yes', MultipleLines='Yes', InternetService='Fiber optic', OnlineSecurity='No', OnlineBackup='Yes', DeviceProtection='Yes', TechSupport='No', StreamingTV='Yes', StreamingMovies='Yes', Contract='One year', PaperlessBilling='Yes', PaymentMethod='Credit card (automatic)', MonthlyCharges=103.2, TotalCharges='7362.9', Churn='No'),
 Row(customerID='4801-JZAZL', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='Yes', tenure=11, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineSecurity='Yes', OnlineBackup='No', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.6, TotalCharges='346.45', Churn='No'),
 Row(customerID='8361-LTMKD', gender='Male', SeniorCitizen=1, Partner='Yes', Dependents='No', tenure=4,